In [29]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
import mxnet as mx
from mxnet import nd, init, gluon, autograd, image
from mxnet.gluon import data as gdata, loss as gloss, nn
import numpy as np
import d2l
import dense_VAE_model
from mxnet.gluon.model_zoo import vision

In [35]:
resnet18

ResNetV2(
  (features): HybridSequential(
    (0): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=True, use_global_stats=False, in_channels=3)
    (1): Conv2D(3 -> 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (2): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=64)
    (3): Activation(relu)
    (4): MaxPool2D(size=(3, 3), stride=(2, 2), padding=(1, 1), ceil_mode=False)
    (5): HybridSequential(
      (0): BasicBlockV2(
        (bn1): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=64)
        (conv1): Conv2D(64 -> 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=64)
        (conv2): Conv2D(64 -> 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (1): BasicBlockV2(
        (bn1): BatchNorm(axis=1, eps=1e

In [49]:
resnet18 = vision.resnet18_v2(pretrained=True)
ctx = d2l.try_gpu()

In [54]:
resnet18_tail = resnet18.features[:9]

# Modify the output layer such that the pretrained model can perform 
# binary classification
head = nn.HybridSequential(prefix='head')
head.add(nn.Dense(256, weight_initializer=init.Xavier, activation='relu'))
head.add(nn.Dropout(0.5))
head.add(nn.Dense(128, weight_initializer=init.Xavier, activation='relu'))
head.add(nn.Dropout(0.5))
head.add(nn.Dense(64, weight_initializer=init.Xavier, activation='relu'))
head.add(nn.Dropout(0.5))
head.add(nn.Dense(1, weight_initializer=init.Xavier))

binary_resnet18 = nn.HybridSequential()
binary_resnet18.add(resnet18_tail)
binary_resnet18.add(head)

In [56]:
binary_resnet18

HybridSequential(
  (0): HybridSequential(
    (0): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=True, use_global_stats=False, in_channels=3)
    (1): Conv2D(3 -> 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (2): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=64)
    (3): Activation(relu)
    (4): MaxPool2D(size=(3, 3), stride=(2, 2), padding=(1, 1), ceil_mode=False)
    (5): HybridSequential(
      (0): BasicBlockV2(
        (bn1): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=64)
        (conv1): Conv2D(64 -> 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=64)
        (conv2): Conv2D(64 -> 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (1): BasicBlockV2(
        (bn1): BatchNorm(axis=1, eps=1

In [ ]:
def train_D(Distriminator, train_loader, val_loader, config):
    num_epoch, ctx = config['num_epoch'], config['ctx']
    loss, optimizer = config['loss'], config['optimizer']
    trainer = gluon.Trainer(Distriminator.collect_params(), optimizer, {'learning_rate': 0.01})
    train_loss, val_loss = [], []
    for epoch in range(1, num_epoch+1):
        for i, (train_x, train_y) in enumerate(train_loader):
            train_x = train_x.as_in_context(ctx)
            train_y = train_y.as_in_context(ctx)
            with autograd.record():
                output = Distriminator(train_x)
                pred = nd.sigmoid()